# Preparation

Import required dependencies to use Google's API

In [1]:
import os.path

from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

Define the accesses to be granted

In [2]:
SCOPES = ['https://www.googleapis.com/auth/classroom.courses.readonly', # View your Google Classroom classes.
          'https://www.googleapis.com/auth/classroom.rosters', # Manage your Google Classroom class rosters.
          'https://www.googleapis.com/auth/classroom.profile.emails', # View the email addresses of people in your classes.
          'https://www.googleapis.com/auth/classroom.topics', #See, create, and edit topics in Google Classroom.
          'https://www.googleapis.com/auth/classroom.coursework.students', # Manage coursework and grades for students
          'https://www.googleapis.com/auth/classroom.courseworkmaterials', # See, edit, and create classwork materials in Google Classroom.
          'https://www.googleapis.com/auth/spreadsheets.readonly'] # See all your Google Sheets spreadsheets.

Use the provided `token.json` file to authenticate and access user data, if `token.json` is not available, it would be created from the provided `credentials.json` by logging into user's Google Account

In [3]:
creds = None

if os.path.exists('token.json'):
    creds = Credentials.from_authorized_user_file('token.json', SCOPES)
# If there are no (valid) credentials available, let the user log in.
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json', SCOPES)
        creds = flow.run_local_server(port=0)
    # Save the credentials for the next run
    with open('token.json', 'w') as token:
        token.write(creds.to_json())

# Score Academy

In the section, the user is going to access the scores that have been entered on the spreadsheet `Score Academy`

Input the link to the Score Academy spreadsheet and retrieve the Spreadsheet ID

In [4]:
SCORE_ACADEMY_LINK = 'https://docs.google.com/spreadsheets/d/1cGJ0pn9k9gKCBnceWVwaL9D7BBDMNjLh8uPYlaBlJi8/edit?usp=sharing'
SCORE_ACADEMY_ID = SCORE_ACADEMY_LINK.split(sep='/')[-2]

Specify the sheet and the cell ranges that is going to be accessed

In [5]:
NAMA_SHEET = "Academy: Batch 23" # Wizard
GRADE_RANGE = [f'{NAMA_SHEET}!D:D', f'{NAMA_SHEET}!E:G', f'{NAMA_SHEET}!L:Q']

Call the Google Spreadsheet API and retrieve the values of the ranges that have been specified

In [6]:
try:
    service = build('sheets', 'v4', credentials=creds)

    sheet = service.spreadsheets().values().batchGet(spreadsheetId=SCORE_ACADEMY_ID,
                                                     ranges=GRADE_RANGE).execute()
    values = sheet.get('valueRanges', [])
        
except HttpError as error:
    print(error)

Concat the retrieved values as a dataframe

In [7]:
import pandas as pd

email = pd.DataFrame(values[0].get('values'))
grade_dv = pd.DataFrame(values[1].get('values'))
grade_ml = pd.DataFrame(values[2].get('values'))

df = pd.concat([email, grade_dv, grade_ml], axis=1)
df.columns = df.iloc[0]
df.drop(index=0, inplace=True)

df.head(5)

,Email,P4DS-PS Quiz,DV Quiz,IP Quiz,RM Quiz,C1 Quiz,C2 Quiz,UL Quiz,TS Quiz,NN Quiz
1,danadyaksawijaya@gmail.com,4,None,None,NaN,NaN,NaN,NaN,NaN,NaN
2,berlianahar@gmail.com,4,None,None,NaN,NaN,NaN,NaN,NaN,NaN
3,angiegrace35@gmail.com,4,None,None,NaN,NaN,NaN,NaN,NaN,NaN
4,ihsan.azmi.rabbani@gmail.com,4,None,None,NaN,NaN,NaN,NaN,NaN,NaN
5,lhpratama2@gmail.com,4,None,None,NaN,NaN,NaN,NaN,NaN,NaN


# Course
In this section, the user is going to choose which Google Classroom Course that is going to be accessed

Call the Google Classroom API to access various methods with user's access from the credential that has been authenticated

In [8]:
service = build('classroom', 'v1', credentials=creds)

Use the `courses().list()` method to show a list of the user's courses

In [9]:
results = service.courses().list(pageSize=5).execute()
courses = results.get('courses', [])

if not courses:
    print('No courses found.')

# Prints the names of the first 5 courses.
print('Courses:')
for course in courses:
    print(f"- {course['name']}")

Courses:
- Wizard Data Visualization
- Kementerian BUMN
- Automation
- Wizard Data Analytics
- Vulcan Machine Learning


Input the course name (**case-insensitive**) that is going to be accessed (Example: `wizard data visualization`). If the course was found, the course's `id` will be retrieved.

In [10]:
course_input = input("Enter Google Classroom Class Name : ")
course_lowercase = course_input.lower()
course_id = None

for course in courses:
    if course_lowercase == course['name'].lower():
        course_id = course['id']
        break

if course_id == None:
    raise Exception(f"{course_input} course not found")

else:
    print(f'{course_input} found with ID {course_id}')

wizard data visualization found with ID 621398536690


# Quiz
In this section, the grading process is going to be done using the score retrieved from the spreadsheet

Input the quiz code (**case-insensitive**) that is going to be accessed. If the quiz was found, the quiz's `id` will be retrieved. **Quiz Input :**

 - `'P4DS'` : 1. Q: Programming for Data Science (P4DS) & Practical Statistic (PS)
 - `'DV'` : 2. Q: Data Visualization (DV)
 - `'IP'` : 3. Q: Interactive Plotting (IP)

In [11]:
from assets import classcode

service = build('classroom', 'v1', credentials=creds)
response = service.courses().courseWork().list(courseId=course_id).execute()
classworks = response.get('courseWork')

while response.get('nextPageToken'):
    response = service.courses().students().list(courseId=course_id, pageToken = response['nextPageToken']).execute()
    classworks.extend(response.get('courseWork'))

quiz_input = input("Enter Quiz Code: ")
quiz_id = None

for classwork in classworks:
    if classwork['title'] == classcode(quiz_input):
        quiz_id = classwork['id']
        break

if quiz_id == None:
    raise Exception(f"Quiz not found")
else:
    print(f"{classwork['title']} Quiz was found")


1. Q: Programming for Data Science (P4DS) & Practical Statistic (PS) Quiz was found


### Draft Grades

Submissions from the chosen quiz are going to be graded as **Draft Grade** 

Use the `courses().courseWork().studentSubmissions().list()` method to store a list of the quiz's submissions

In [14]:
submissions = []

response = service.courses().courseWork().studentSubmissions().list(
    courseId=course_id,
    courseWorkId=quiz_id).execute()
submissions.extend(response.get('studentSubmissions', []))

while response.get('nextPageToken'):
    response = service.courses().courseWork().studentSubmissions().list(
        courseId=course_id,
        courseWorkId=quiz_id,
        pageToken = response['nextPageToken']).execute()
    submissions.extend(response.get('studentSubmissions'))

In [15]:
len(submissions)

57

All the stored submission are graded as draft in accordance with the student's e-mail in the Google Classroom using `courses().courseWork().studentSubmissions().patch()` method

In [16]:
from assets import quizcode

draft = []
quiz_code = quizcode(quiz_input)

for submission in submissions:
  # Retrieve student's email
  submission_profile = service.courses().students().get(courseId=course_id, userId=submission['userId']).execute()
  student_df = df.loc[df['Email'] == submission_profile['profile']['emailAddress']]
  
  # Retrieve student's grade
  if not student_df.empty:
    if student_df[quiz_code].values[0].isnumeric():
      submission_grade = student_df[quiz_code].values[0]
    else:
      print(f"WARNING: {submission_profile['profile']['name']['fullName']} ({submission_profile['profile']['emailAddress']}) has no grade")
      draft.append([submission_profile['profile']['name']['fullName'], None, "NO GRADE"])
      continue      
  else:
    print(f"WARNING: {submission_profile['profile']['name']['fullName']} ({submission_profile['profile']['emailAddress']}) was not found")
    draft.append([submission_profile['profile']['name']['fullName'], None, "NOT FOUND"])
    continue

  # Grade the submission as draftGrade
  # studentSubmission = {
  #   'draftGrade': str(submission_grade)
  # }

  # response = service.courses().courseWork().studentSubmissions().patch(
  #     courseId=course_id,
  #     courseWorkId=classwork['id'],
  #     id=submission['id'],
  #     updateMask='draftGrade',
  #     body = studentSubmission ).execute()
  
  draft.append([submission_profile['profile']['name']['fullName'], submission_grade, "GRADED"])

draft_df = pd.DataFrame(draft, columns=['Name', 'Grade', 'Status'])
draft_df

,Name,Grade,Status
0,Nissa Corlidea,4,GRADED
1,Cak Mustafa (cakmustafa),4,GRADED
2,Shavira Zhalsabilla,4,GRADED
3,Dea,4,GRADED
4,Albertus MPP,4,GRADED
5,Taufiq Rahman,4,GRADED
6,Pramadia Adhie Lazuardi,4,GRADED
7,Bintang Setyawan,0,GRADED
8,Fajar khamim,4,GRADED
9,Grace Angela Sutantya,4,GRADED


Confirm Draft Grade before continuing

In [14]:
confirmation = input('Return grades? [y]/[n] : ')
if confirmation.lower() == 'y':
    print("Returning student's grades")
else :
    raise Exception("Return cancelled")

Returning student's grades


### Return Grades

Submissions from the chosen quiz are going to be graded as **Assigned Grade**

All the stored submission are grades are **returned** in accordance with the student's e-mail in the Google Classroom using `courses().courseWork().studentSubmissions().patch()` method

In [15]:
try:
    for submission in submissions:
        # Retrieve student's email
        submission_profile = service.courses().students().get(courseId=course_id, userId=submission['userId']).execute()
        student_df = df.loc[df['Email'] == submission_profile['profile']['emailAddress']]
        
        # Retrieve student's grade
        if not student_df.empty:
            if student_df[quiz_code].values[0].isnumeric():
                submission_grade = student_df[quiz_code].values[0]
            else:
                continue
        else:
            continue
        
        # Grade the submission as assignedGrade
        studentSubmission = {
            'assignedGrade': str(submission_grade)
        }

        response = service.courses().courseWork().studentSubmissions().patch(
            courseId=course_id,
            courseWorkId=classwork['id'],
            id=submission['id'],
            updateMask='assignedGrade',
            body = studentSubmission).execute()
        
    print('Quiz grades was successfully returned')

except HttpError as error:
    print(error)

Quiz grades was successfully returned
